In [16]:
from db.duckdb.duckdbhelper import DuckDBDatabaseHelper
from pprint import pprint
import pandas as pd

In [17]:
NUM_ROWS = 1500000

In [18]:
db = DuckDBDatabaseHelper("meters.db")

In [19]:
sql_query ="""

SELECT day,energy_sum, 
temperatureMin, 
temperatureMax,
uvIndex,
dewPoint,                  
Type
FROM  meters m LEFT JOIN info_household ih
ON m.LCLid = ih.LCLid
LEFT JOIN weather_daily_darksky_modified wd
ON m.day = wd.formatted_temperatureMaxTime_weather_daily_darksky
LEFT JOIN uk_bank_holidays uk_bh
ON m.day = uk_bh."Bank holidays"

WHERE date_part('year', day) = 2013

"""

db.connect()
records = db.fetch_all(sql_query)
db.close_connection()

Connected to DuckDB database: meters.db
Fetched 1966727 rows.
Connection closed.


In [20]:
records[:5]

[(datetime.date(2013, 1, 3), 10.074, 9.65, 11.41, 1.0, 9.12, None),
 (datetime.date(2013, 1, 6), 10.293000000000003, 6.91, 8.61, 1.0, 7.0, None),
 (datetime.date(2013, 1, 7), 9.439000000000002, 7.1, 9.08, 1.0, 6.78, None),
 (datetime.date(2013, 1, 11), 10.978999900000002, 0.89, 5.7, 1.0, 1.68, None),
 (datetime.date(2013, 1, 12), 10.585999900000003, 1.75, 4.53, 1.0, 0.79, None)]

In [21]:
COLS = [
"day","energy_sum", 
"temperatureMin", 
"temperatureMax",
"uvIndex",
"dewPoint",                  
"Type" ]

In [22]:
df_model = pd.DataFrame(records,columns = COLS)

In [23]:
df_model.head()

,day,energy_sum,temperatureMin,temperatureMax,uvIndex,dewPoint,Type
0,2013-01-03,10.074,9.65,11.41,1.0,9.12,None
1,2013-01-06,10.293,6.91,8.61,1.0,7.00,None
2,2013-01-07,9.439,7.10,9.08,1.0,6.78,None
3,2013-01-11,10.979,0.89,5.70,1.0,1.68,None
4,2013-01-12,10.586,1.75,4.53,1.0,0.79,None


In [24]:
COLS_FEAT = ["temperatureMin", 
"temperatureMax",
"uvIndex",
"dewPoint",                  
"Type"]
COLS_LABEL = "energy_sum"

In [25]:
df_model_sample = df_model.sample(n = NUM_ROWS)

In [26]:
X = df_model_sample[COLS_FEAT]

In [27]:
y = df_model_sample[COLS_LABEL]

In [28]:
X.head()

,temperatureMin,temperatureMax,uvIndex,dewPoint,Type
1675501,11.33,16.81,4.0,9.65,None
953304,11.88,24.94,3.0,12.37,None
1543162,2.14,8.99,2.0,0.86,None
278575,14.92,25.17,5.0,13.17,None
468025,2.33,6.11,1.0,0.84,None


In [29]:
y

1675501    11.129
953304      1.951
1543162     5.391
278575     17.784
468025     24.062
            ...  
1735345     3.956
1250259    14.058
573339      1.516
132430      5.736
695992      2.784
Name: energy_sum, Length: 1500000, dtype: float64

In [30]:
df_model_sample.to_csv("train.csv",index = False)